In [11]:
import shutil
from pathlib import Path
from xml.etree import cElementTree as ET
import pandas as pd

In [2]:
class XmlParser:
    def __init__(self, xml_file):
        self._xml_tree = ET.parse(xml_file)
        
    def base_count(self, qscore=0):
        count = 0
        for qual in self._xml_tree.findall('*/Quality'):
            if int(qual.attrib['value']) >= qscore:
                count += int(qual.attrib['count'])
        return count
    
    @property
    def layout(self):
        return self._xml_tree.find('Statistics').attrib['nreads']
    
    @property
    def length(self):
        return tuple(map(lambda x: int(x.attrib['average']), self._xml_tree.findall('*//Read')))

In [16]:
dirpath = Path('/home/chen1i6c04/year2/xml')

data = []
for xml in dirpath.iterdir():
    parser = XmlParser(xml)
    data.append((xml.stem, parser.layout, parser.base_count(30)/parser.base_count()*100))

In [38]:
df = pd.DataFrame(data, columns=['Run', 'Layout', 'Q30%'])

In [43]:
df['Q30%'] = df['Q30%'].round(2)

In [39]:
df['FilterPass'] = (df['Q30%']>=80)&(df['Layout']=='2')

In [44]:
df.to_csv('/home/chen1i6c04/year2/QC.tsv', sep='\t', index=False)

In [46]:
df.shape

(362308, 4)

In [50]:
df[df['FilterPass']!=True].shape

(37472, 4)

In [51]:
df[df['FilterPass']==True].shape

(324836, 4)